In [ ]:
from s3i import UserMessage,IdentityProvider, TokenType, Broker, Key, Directory
import os
import uuid
import base64
import getpass

In [ ]:
USERNAME = None
PASSWORD = getpass.getpass('[S3I]: Please enter the password:')

In [ ]:
CLIENT = None
SECRET = getpass.getpass('[S3I]: Please enter the secret:')  

In [ ]:
IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)

# Get Token


In [ ]:
print("[S3I]: Access token: \n ", IDP.get_token(TokenType.ACCESS_TOKEN))

# Create HMI

In [ ]:
HMI = None 
HMI_SECRET = None

# Add Public Key

In [ ]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
response = dir.putPublicKey(HMI,"key/workshop_pub.asc")
print(response.text)

# Send User Messages

## Verify the endpoints

In [ ]:
receivers = [str(x) for x in input('[S3I]: Please enter the receivers, seperate with a space:').split()]

In [ ]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
defaultHMIs = [dir.queryThingIDBased(receiver+"/attributes/defaultHMI") for receiver in receivers]
endpoints = [dir.queryThingIDBased(hmi+"/attributes/allEndpoints")[0] for hmi in defaultHMIs]
print("Endpoints:" , endpoints)

In [ ]:
endpoints = [str(x) for x in input('[S3I]: Please enter the receivers, seperate with a space:').split()]

## Prepare the message

In [ ]:
subject = input('[S3I]: please enter the subject: \n')
text = input('[S3I]: please enter the text: \n')
msg_uuid = "s3i:" + str(uuid.uuid4())
replyToEndpoint=None
attachments=None

### Add attachement to message

In [ ]:
filename_list = list(map(str, input("[S3I]: Please enter the name of file that you want to send: ").split()))
attachments = list()

for filename in filename_list:
    attachment_dict = dict()
    attachment_dict["filename"] = filename
    with open("attachment_files/"+filename, "rb") as f:
        # encode an attachment file to BASE64 bytes
        base64_data = base64.b64encode(f.read())
        data = base64_data.decode()  # convert byte to str
        attachment_dict["data"] = data
        attachments.append(attachment_dict)

## Assemble User Message

In [ ]:
message = UserMessage(sender=CLIENT, identifier=msg_uuid,receivers=receivers, subject=subject, text=text,
                  replyToEndpoint=replyToEndpoint, attachments=attachments)
print(message.msg)

## Sign the message

In [ ]:
pgpKey_sec = Key(path_demo_dir="key/", filename="workshop_sec.asc")
message.sign(pgpKey_sec.key)
#print(message.pgpMsg.__str__())

## Encrypt the message

In [ ]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
publicKeys_str = dir.getPublicKey(defaultHMIs)
publicKeys = [Key(key_str=i) for i in publicKeys_str]
message.encrypt(publicKeys)
print(message.pgpMsg.__str__())

## Send the message via S³I Broker

In [ ]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
broker.send(endpoints, message.pgpMsg.__str__())